In [1]:
from text2vec import SentenceModel
import mysql.connector
import faiss
import numpy as np
import json

# 加载模型
model = SentenceModel("BAAI/bge-large-zh-v1.5")  



2025-05-30 12:08:31.642 | DEBUG    | text2vec.sentence_model:__init__:78 - Use pytorch device: cuda


In [2]:
# 读取数据库中的题目数据
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='root',
    database='chatbot'
)
cursor = conn.cursor(dictionary=True)
cursor.execute("SELECT title, description FROM problem")
rows = cursor.fetchall()

In [3]:
from bs4 import BeautifulSoup
import html
import re

def clean_description(html_content: str) -> str:
    soup = BeautifulSoup(html_content, "html.parser")
    for tag in soup(["style", "script"]):
        tag.decompose()
    text = soup.get_text(separator="\n")
    text = html.unescape(text)
    text = re.sub(r'\s+', ' ', text)  
    text = re.sub(r'(\n\s*)+', '\n', text)  
    return text.strip()

In [4]:
for q in rows:
    raw_html = q.get("description", "")
    cleaned_text = clean_description(raw_html)
    q["description"] = cleaned_text

In [5]:
# 构建标题和描述向量
titles = [r["title"] for r in rows]
descs = [r["description"] for r in rows]
title_vecs = model.encode(titles, normalize_embeddings=True)
desc_vecs = model.encode(descs, normalize_embeddings=True)

In [6]:
# 建立FAISS索引
dim = title_vecs.shape[1]
title_index = faiss.IndexFlatIP(dim)
desc_index = faiss.IndexFlatIP(dim)

title_index.add(np.array(title_vecs))
desc_index.add(np.array(desc_vecs))

# 保存索引和映射
faiss.write_index(title_index, "title.index")
faiss.write_index(desc_index, "desc.index")

with open("index2title.json", "w", encoding="utf-8") as f:
    json.dump(titles, f, ensure_ascii=False)